In [1]:
import json 
import pandas as pd
import numpy as np
import random as rand

In [2]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np 
import regex as re 

In [3]:
with open('./Data/masterchef_data_cleaned.json', 'r') as file:
    data = json.load(file)
# data

In [4]:
sentences = []

In [5]:
country_list = ['United States', 'Canada', 'Brazil']
max_ssn = [13, 7, 10]
seasons = []
for r in max_ssn:
    ssn_prep = []
    for i in range(r):
        ssn_prep.append([])
    seasons.append(ssn_prep)


In [6]:
for r, country in enumerate(country_list):
    for i in range(len(data[country])):
        data[country][i]['season']
        seasons[r][data[country][i]['season']-1].append(data[country][i]['evaluations'])
        # print(data['United States'][i]['evaluations'])
        sent_i = data[country][i]['evaluations']
        sentences.append(sent_i)

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

In [145]:
blocks = []
for country in range(len(seasons)):
    for season in range(len(seasons[country])):
        ssn_block = []
        for contestant in range(len(seasons[country][season])):
            ssn_block.append(seasons[country][season][contestant])
        blocks.append(ssn_block)                         
        

In [148]:
test_blocks = []
for i in range(8):
    rdm = rand.randint(0,len(blocks)-1)
    test_blocks.append(blocks[rdm])
    blocks.pop(rdm)

In [149]:
len(test_blocks)

8

In [151]:
input_sequences = []
for b in range(len(blocks)):
    for i in range(len(blocks[b])):
        token_list = tokenizer.texts_to_sequences([blocks[b][i]])[0]
        for j in range(1, len(token_list)): 
            n_gram_sequence = token_list[:j+1] 
            input_sequences.append(n_gram_sequence)
            

In [152]:
test_input_sequences = []
for b in range(len(test_blocks)):
    for i in range(len(test_blocks[b])):
        token_list = tokenizer.texts_to_sequences([test_blocks[b][i]])[0]
        for j in range(1, len(token_list)): 
            n_gram_sequence = token_list[:j+1] 
            test_input_sequences.append(n_gram_sequence)
            

In [157]:
# Pad sequences and split into predictors and label 
max_sequence_len = max([len(seq) for seq in input_sequences]) 
max_sequence_len_test = max([len(seq) for seq in test_input_sequences])
if max_sequence_len_test > max_sequence_len:
    max_sequence_len = max_sequence_len_test
input_sequences = np.array(pad_sequences( 
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [158]:
test_input_sequences = np.array(pad_sequences( 
    test_input_sequences, maxlen=max_sequence_len, padding='pre')) 
X_test, y_test = test_input_sequences[:, :-1], test_input_sequences[:, -1]

In [159]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words) 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=total_words) 


In [160]:
X.shape

(5066, 39)

In [161]:
X_test.shape

(1804, 39)

In [162]:
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [164]:
X.shape

(5066, 39)

In [165]:
y.shape

(5066, 17)

In [166]:
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
159/159 [==============================] - 6s 27ms/step - loss: 2.1645 - accuracy: 0.3064
Epoch 2/20
159/159 [==============================] - 5s 33ms/step - loss: 2.0734 - accuracy: 0.3216
Epoch 3/20
159/159 [==============================] - 5s 33ms/step - loss: 2.0350 - accuracy: 0.3423
Epoch 4/20
159/159 [==============================] - 5s 30ms/step - loss: 2.0119 - accuracy: 0.3413
Epoch 5/20
159/159 [==============================] - 5s 34ms/step - loss: 1.9597 - accuracy: 0.3549
Epoch 6/20
159/159 [==============================] - 5s 30ms/step - loss: 1.9129 - accuracy: 0.3772
Epoch 7/20
159/159 [==============================] - 5s 33ms/step - loss: 1.8851 - accuracy: 0.3829
Epoch 8/20
159/159 [==============================] - 4s 27ms/step - loss: 1.8726 - accuracy: 0.3867
Epoch 9/20
159/159 [==============================] - 3s 16ms/step - loss: 1.8578 - accuracy: 0.3857
Epoch 10/20
159/159 [==============================] - 3s 16ms/step - loss: 1.8516 - accura

In [167]:
preds = model.predict(X_test)
preds = [tokenizer.index_word[np.argmax(row)] for row in preds]
y_test = [tokenizer.index_word[np.argmax(row)] for row in y_test]

57/57 [==============================] - 1s 6ms/step


In [134]:
y_test

['high_i',
 'win_t',
 'in_i',
 'high_i',
 'win_t',
 'low_t',
 'low_i',
 'high_i',
 'in_i',
 'high_i',
 'low_t',
 'win_i',
 'win_i',
 'in_i',
 'high_i',
 'win_i',
 'in_i',
 'win_i',
 'in_i',
 'win_i',
 'win_i',
 'winner',
 'in_i',
 'win_t',
 'low_t',
 'in_i',
 'win_t',
 'in_i',
 'high_i',
 'win_i',
 'high_i',
 'low_i',
 'high_i',
 'in_i',
 'win_t',
 'in_i',
 'low_i',
 'win_t',
 'high_i',
 'low_i',
 'win_i',
 'imm',
 'low_i',
 'high_i',
 'win_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'high_i',
 'low_i',
 'low_i',
 'low_i',
 'win_i',
 'imm',
 'low_i',
 'in_i',
 'win_i',
 'win_t',
 'in_i',
 'low_i',
 'high_i',
 'low_i',
 'elim',
 'in_i',
 'win_t',
 'low_t',
 'low_i',
 'low_i',
 'high_i',
 'imm',
 'low_i',
 'in_i',
 'win_t',
 'low_t',
 'in_i',
 'win_t',
 'high_i',
 'imm',
 'win_t',
 'win_i',
 'imm',
 'in_i',
 'elim',
 'in_i',
 'win_t',
 'in_i',
 'high_i',
 'win_t',
 'high_i',
 'imm',
 'win_t',
 'in_i',
 'win_t',
 'in_i',
 'high_i',
 'win_t',
 'in_i',
 'high_i',
 'low_i',
 'in_i',
 'elim',
 '

In [168]:
run_cor = 0
run_elim = 0
for i in range(len(y_test)):
    print(y_test[i], '\t', preds[i])
    if y_test[i] == 'elim':
        run_elim += 1
    if y_test[i] == preds[i]:
        run_cor += 1

imm_2 	 in_i
win_t 	 win_t
high_i 	 in_i
win_i 	 imm
low_i 	 win_t
win_i 	 in_i
imm 	 imm
win_i 	 win_t
win_i 	 in_i
imm 	 imm
imm 	 win_t
imm_2 	 in_i
high_i 	 in_i
low_t 	 imm
win_t 	 in_i
high_i 	 in_i
win_i 	 imm
win_i 	 in_i
imm 	 imm
winner 	 in_i
win_i 	 imm
imm 	 win_t
in_i 	 in_i
in_i 	 in_i
win_t 	 in_i
in_i 	 in_i
in_i 	 in_i
win_i 	 win_t
in_i 	 in_i
in_i 	 in_i
imm 	 in_i
in_i 	 in_i
high_i 	 in_i
win_t 	 win_t
win_i 	 in_i
win_i 	 imm
imm 	 imm
in_i 	 in_i
win_i 	 in_i
runner_up(s) 	 imm
in_i 	 in_i
in_i 	 win_t
in_i 	 in_i
win_i 	 in_i
in_i 	 imm
in_i 	 in_i
in_i 	 in_i
win_t 	 in_i
high_i 	 in_i
in_i 	 imm
imm 	 in_i
in_i 	 in_i
win_i 	 in_i
imm 	 in_i
low_i 	 in_i
in_i 	 in_i
high_i 	 elim
in_i 	 in_i
elim 	 in_i
in_i 	 in_i
win_t 	 win_t
win_i 	 in_i
low_i 	 imm
in_i 	 win_t
in_i 	 in_i
in_i 	 in_i
elim 	 win_t
ret 	 in_i
low_i 	 in_i
in_i 	 in_i
win_t 	 in_i
win_t 	 in_i
high_i 	 in_i
low_i 	 in_i
elim 	 in_i
in_i 	 in_i
win_i 	 win_t
in_i 	 in_i
in_i 	 in_i
win_i 	 

In [169]:
run_cor / len(y_test)

0.3553215077605322

In [170]:
run_elim / len(y_test)

0.07039911308203992

In [123]:
preds

['in_i',
 'imm',
 'in_i',
 'in_i',
 'in_i',
 'imm',
 'in_i',
 'in_i',
 'in_i',
 'imm',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'win_t',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'win_t',
 'in_i',
 'in_i',
 'in_i',
 'elim',
 'elim',
 'in_i',
 'imm_2',
 'in_i',
 'in_i',
 'imm',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'imm',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'high_i',
 'in_i',
 'in_i',
 'in_i',
 'elim',
 'elim',
 'in_i',
 'imm',
 'win_t',
 'in_i',
 'win_t',
 'in_i',
 'win_t',
 'win_t',
 'win_t',
 'win_t',
 'win_t',
 'win_t',
 'in_i',
 'win_t',
 'in_i',
 'win_t',
 'in_i',
 'win_t',
 'low_t',
 'in_i',
 'elim',
 'elim',
 'elim',
 'elim',
 'imm',
 'in_i',
 'in_i',
 'imm',
 '4a86e8-2nd',
 'win_t',
 'in_i',
 'in_i',
 'in_i',
 'in_i',
 'high_t',
 'in_i',
 'imm',
 '

In [144]:
seed_text = 'HIGH_I LOW_T LOW_I'
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=39, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("\nOutput: ", seed_text)

1/1 [==============================] - 0s 20ms/step

Output:  HIGH_I LOW_T LOW_I imm in_i in_i in_i in_i
